In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Convolution2D, Flatten, Dropout,GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras import models
from keras import layers
from keras import optimizers
from keras import metrics
from keras.applications.inception_resnet_v2 import preprocess_input
from attention import AttentionLayer
from sklearn.metrics import roc_auc_score
import os
import pandas as pd
#import keras
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

In [ ]:
datadir = "./data/akac_gen.txt"
dataset = pd.read_csv(datadir, delimiter ="\t", header = 0)
#print(list(dataset.columns))
dataset = dataset.sample(frac=1, random_state=777)  # row 전체 shuffle

SNPs = ['rs2235373','rs16873348','rs2284791','rs2013162','rs595918','rs3753582','rs2235375','rs1044516','rs11204737','rs1044516','rs7715100','rs3917192','rs2235371']
full = list(dataset.columns)
del full[0:6]
print(full)
snps = ['rs11204737', 'rs2255796', 'rs2235373','rs16873348','rs2013162', 'rs1139','rs2284791','rs595918','rs3753582']
#
#['rs2235373','rs16873348','rs2284791','rs2013162','rs595918','rs3753582']
#dataset.head()

In [ ]:
dataset.shape
y_train = pd.to_numeric(dataset['affected']) -1
X_train = dataset[snps]
x_train_np = np.array(X_train).astype("float")
y_train_np = np.array(y_train)
input_dim = X_train.shape[1]
print(x_train_np.shape)

In [ ]:
snps_num = 10
no_of_generations = 30
no_of_individuals = 18
mutate_factor = 0.02

In [ ]:
def mutate(new_individual):
    genes = []
    leftover = [v for v in full if v not in new_individual]
    for gene in new_individual:
        n = random.random()
        if(n < mutate_factor):
            k = random.randrange(len(leftover))
            genes.append(leftover[k])
            del leftover[k]
        else:
            genes.append(gene)  
    return genes

def crossover(individuals):
    new_individuals = []
    elite = 6
    for i in range(elite):
        new_individuals.append(individuals[i])
    #엘리트 살아남음

    for i in range(elite, no_of_individuals):
        new_individual = []
        #if(i < (no_of_individuals - 2)):
        if(i == 2):
            parentA = random.choice(individuals[:3])
            parentB = random.choice(individuals[:3])
        else:
            parentA = random.choice(individuals[:])
            parentB = random.choice(individuals[:])
        #아래에서 유전자 반반씩 나눠 줌
        for i in range(len(parentA)):
            n = random.random()
            if(n< 0.5):
                if parentA[i] in new_individual:
                    new_individual.append(parentB[i])
                else:
                    new_individual.append(parentA[i])
            else:
                if parentB[i] in new_individual:
                    new_individual.append(parentA[i])
                else:
                    new_individual.append(parentB[i])
        #else:
        #    new_individual = random.choice(individuals[:])

        new_individuals.append(mutate(new_individual))
        #new_individuals.append(new_individual)

    return new_individuals

def evolve(individuals, fitness):
  
    sorted_y_idx_list = sorted(range(len(fitness)),key=lambda x:fitness[x])
    individuals = [individuals[i] for i in sorted_y_idx_list ]
    fitness = [fitness[i] for i in sorted_y_idx_list ]
    print(fitness)
    #individuals.reverse()

    new_individuals = crossover(individuals)

    return new_individuals

In [ ]:
def load_model(size, input_dim):
    model = Sequential()
    model.add(Dense(size,input_dim = input_dim,activation='relu'))
    model.add(Dense(size*2,input_dim = input_dim,activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(size*2,input_dim = size,activation='relu'))
    model.add(Dense(1,input_dim = size,activation='sigmoid', name='lastfc'))
    opt = optimizers.Adam(lr=5e-3,decay=1e-5)
    model.compile(loss='mse', optimizer=opt, metrics=["accuracy"])
    
    return model

In [ ]:
def train(individuals):
    fitness = []
    fitness_a = []
    for i in range(len(individuals)):
        model.set_weights(Wsave)
        X_train = dataset[individuals[i]]
        x_train_np = np.array(X_train).astype("float")
        
        history = model.fit(x=X_train,y=y_train, epochs=80,verbose=0,validation_split=0.25)
        
        for k,j in zip(history.history["val_loss"], history.history["loss"]):
            fitness_a.append((k+j)/2)  
        fitness.append(round(min(fitness_a), 4))
        
    return individuals, fitness

In [ ]:
individuals = []
for i in range(no_of_individuals):
    individuals.append(random.sample(SNPs, snps_num))
    
# control loop
size = 16
model = load_model(size, snps_num)
Wsave = model.get_weights()
best = []
best_fitness = 10
best_generation = 0

for generation in range(no_of_generations):
    
    individuals, fitness = train(individuals)
    print(fitness)
    
    if best_fitness >= min(fitness):
        best = individuals[fitness.index(min(fitness))]
        best_fitness = min(fitness)
        best_generation = generation+1
        
    if generation == no_of_generations -1:
        break       
    individuals = evolve(individuals, fitness)

In [ ]:
print(best)
print(best_fitness)
print(best_generation)
print(individuals)

In [ ]:
test_loss, test_acc = model.evaluate(x_train_np, y_train_np, batch_size=20, verbose=2)
predict_test = model.predict(x_train_np, batch_size=20)

print(roc_auc_score(y_train_np, predict_test))

plotResults(history)

from sklearn.metrics import roc_curve, auc
nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y_train_np, predict_test)
auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
plt.plot(nn_fpr_keras, nn_tpr_keras, marker='.', label='Neural Network (auc = %0.3f)' % auc_keras)